In [3]:
import sys
sys.path.append('./src')
import syllo_gen
import random
import json
from tqdm.autonotebook import tqdm
import openai
openai.api_key = open('key.txt').read().strip()

%load_ext autoreload
%autoreload 2

/var/folders/b2/2pp0lc_n7s78sl3jfrdxggb80000gn/T/ipykernel_45511/2018132128.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# ADJ Template

In [7]:
q,v = syllo_gen.get_syllo(3)
# q = syllo_gen.negate_quesion(q)
v = syllo_gen.random_assign_adjs(v)
print(syllo_gen.question2fol(q, v))
print(syllo_gen.question2template(q, v, rand=True, noun=False))

{'story': ['∀x (shrewd(x) → ¬fluent(x))', '∃x (spirited(x) ∧ shrewd(x))', '∀x (charming(x) → fluent(x))', '∀x (mighty(x) → charming(x))'], 'conclusion': '∃x (spirited(x) ∧ ¬mighty(x))'}
{'story': ['Shrewd people are not fluent.', 'Some spirited person is shrewd.', 'There does not exist a charming and not fluent person.', 'There is no mighty person who is not charming.'], 'conclusion': 'Not all spirited people are mighty.'}


# NOUN Template

In [4]:
q,v = syllo_gen.get_syllo(1)
# q = syllo_gen.negate_quesion(q)
v = syllo_gen.random_assign_nouns(v)  # change to correct type
print(syllo_gen.question2fol(q, v))
print(syllo_gen.question2template(q, v, rand=True, noun=True)) # change to correct type


{'story': ['∀x (penalty(x) → ¬community(x))', '∀x (town(x) → community(x))'], 'conclusion': '∀x (town(x) → ¬penalty(x))'}
{'story': ['Penalty is not community for all penalty.', 'All town is community.'], 'conclusion': 'If something is town, then it is not penalty.'}


In [21]:
all_possilbe = set()
prompt = ''
rand = True
for j in range(100):
    if len(all_possilbe) == 24:
        break
    real = random.choice([True, False])
    q, v = syllo_gen.get_syllo(1)
    if not real:
        q = syllo_gen.negate_quesion(q)

    v = syllo_gen.random_assign_nouns(v)
    if tuple(q['story']) not in all_possilbe:
        all_possilbe.add((tuple(q['story'])))
        q = syllo_gen.question2template(q, v, rand=rand)
        prompt += f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}\n'
        prompt += f'Answer: {real}\n\n'

# for j in range(100):
#     if len(all_possilbe) == 30:
#         break
#     real = random.choice([True, False])
#     q, v = syllo_gen.get_syllo(2)
#     if not real:
#         q = syllo_gen.negate_quesion(q)

#     v = syllo_gen.random_assign_nouns(v)
#     if tuple(q['story']) not in all_possilbe:
#         all_possilbe.add((tuple(q['story'])))
#         q = syllo_gen.question2template(q, v, rand=rand)
#         prompt += f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}\n'
#         prompt += f'Answer: {real}\n\n'
print(prompt)

Story: Everything that is garden is also grass. Style is always garden. Question: There is no style that is not grass.
Answer: True

Story: Not a single book is office. Every security is book. Question: If something is security, then it is not office.
Answer: True

Story: Everything that is tree is also voice. There esists some refrigerator that is not voice. Question: Everything that is refrigerator is also tree.
Answer: False

Story: If something is perception, then it is level. Perception is boss for some perception. Question: Some boss is level.
Answer: True

Story: There is no stone that is permission. All part is permission. Question: At lease one part is not stone.
Answer: True

Story: If something is piece, then it is guitar. Guitar is never boat. Question: There esists some boat that is not piece.
Answer: True

Story: Everything that is celebration is also basis. Information is not basis for all information. Question: Information is always celebration.
Answer: False

Story: Th

In [23]:
corret = 0
total= 100
for i in tqdm(range(total)):

    real = random.choice([True, False])
    q, v = syllo_gen.get_syllo(1)
    if not real:
        q = syllo_gen.negate_quesion(q)
    # v = syllo_gen.random_assign_nouns(v)
    q = syllo_gen.question2template(q, v, rand=True)
    tmp_prompt = prompt + f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}'

    tmp_prompt += "\nAnswer:"



    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=tmp_prompt,
    temperature=0,
    max_tokens=10,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop=["\n"]
    )
    text = response['choices'][0]['text'].strip()
    if text not in ['True', 'False']:
        print(text)
        continue
    if text == 'True' and real:
        corret += 1
    elif text == 'False' and not real:
        corret += 1
    else:
        print(f'Story: {" ".join(q["story"])} Question: {q["conclusion"]}')
        print(text)


print(corret/total)

  0%|          | 0/100 [00:00<?, ?it/s]

Story: Everything that is b is also c. There exists some b that is a. Question: A is not c for all a.
True
Story: Every b is c. Every instance of a is also an instance of b. Question: There esists some a that is not c.
True
Story: All c is b. All b is a. Question: A is c for some a.
False
Story: If something is c, then it is b. There is no b that is a. Question: There is no a that is not c.
True
Story: No b is c. Every a is b. Question: There is no a that is not c.
True
Story: B is c for some b. Everything that is b is also a. Question: A is c for some a.
False
Story: C is not b for all c. If something is a, then it is b. Question: A portion of a is also c.
True
0.93


In [ ]:
# TODO: Add grounded statements